In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io
from PIL import Image
import plotly.graph_objects as go
from ipywidgets import interactive, HBox, VBox
import plotly.graph_objs as go
import plotly.offline as py
from plotly.graph_objs import FigureWidget
import plotly.express as px


In [ ]:
top_dir = '/home/ubuntu/bucket/projects'
proj_dir = '2018_11_20_GeneCpdFollowup'
batchname = '2018_11_20_Batch1'
imgplate1 = 'BR00100032__2018-11-20T09_38_24-Measurement2'
imgplate2 = 'BR00100037__2018-11-19T08_19_35-Measurement1'
platename= "BR00100032"

In [ ]:
def make_directory(outpath):
    
    
    if not os.path.exists(outpath):
        
        
        return os.makedirs(outpath)
    
outpath=os.path.join(top_dir, proj_dir, "workspace", "Figures", batchname, platename)
make_directory(outpath)

In [ ]:
class load_data:
    
    def __init__(self,top_dir,proj_dir):
        
        self.top_dir = top_dir
        self.proj_dir = proj_dir
        self.batchname = batchname
    
    
    def csvpath(self):
        plates = ["BR00100032", "BR00100037"]
        
        spath = [os.path.join(self.top_dir, self.proj_dir, "workspace", "backend", self.batchname, p,  p + "_selected_wells.csv") for p in plates]  

        return spath
    
    def imagepath(self):
        plates = [imgplate1, imgplate2]
        imgpath = [os.path.join(self.top_dir, self.proj_dir,self.batchname,'images', p, "Images") for p in plates]
        return imgpath
    
    def featurelist(self):
        fpath = os.path.join(self.top_dir, self.proj_dir,"workspace", "metadata",self.batchname,"featureslist.txt")
        featlist = np.loadtxt(fpath, dtype=str).tolist()
        return featlist
    
    
    
    
subclass = load_data(top_dir, proj_dir)   

csvlist = subclass.csvpath()
imgpath = subclass.imagepath()
featlist = subclass.featurelist()


In [ ]:

umap_path = os.path.join(top_dir, proj_dir, "workspace", "backend", batchname,platename,  platename+"_umap.csv")
                         
data = (pd.read_csv(umap_path)
.drop(columns=['Unnamed: 0'])
       )

                         
data.head() 

In [ ]:
samples = ['DMSO', 'BSJ-04-030', 'BSJ-03-136']

colors = ['gray', 'green', 'red']

label = ['DMSO', 'BSJ-04-030', 'BSJ-03-136']


for s, col, lab in zip(samples, colors, label):

    final = data.loc[data["Metadata_compound_name"] == str(s)]
    
    x, y = final['UMAP-1'], final['UMAP-2'] 
    
    fig = plt.figure(figsize=(10,8))

    plt.scatter(x, y, alpha=0.8, c=col, s=1, label=lab)
    plt.legend(loc=2)
    plt.show()

    


### Rounding locations values and as prescion values more than 4 causes difficulties in quering rows

In [ ]:
def rounding_up(x):
    x['Cells_Location_Center_X'] = round(x['Cells_Location_Center_X'], 4)
    x['Cells_Location_Center_Y'] = round(x['Cells_Location_Center_Y'], 4)
    
    return x


data = rounding_up(data)


#druglist = ['DMSO', 'BSJ-04-030', 'BSJ-03-136']
druglist = ['BSJ-04-030']

varlist = ['Metadata_compound_name', 'Metadata_Well', 'Metadata_Site', 'Cells_Location_Center_X', 'Cells_Location_Center_Y', 'Cells_AreaShape_MajorAxisLength', 'Cells_AreaShape_MinorAxisLength', 'UMAP-1', 'UMAP-2']

df = (data.query("Metadata_compound_name in @ druglist")
      .loc[:, varlist]
      .reset_index()
     )

xaxis='UMAP-1'
yaxis='UMAP-2'

f = go.FigureWidget([go.Scatter(y = df[yaxis], x = df[xaxis], mode = 'markers')])

scatter = f.data[0]
scatter.x = scatter.x 
scatter.y = scatter.y 
#scatter.marker.opacity = 0.4
scatter.marker.size=1
    
    
##Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(
    header=dict(values=varlist,
                fill = dict(color='#7bccc4'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in varlist],
               fill = dict(color='#ccebc5'),
               align = ['left'] * 5))])


axis_dropdowns = interactive(f, yaxis = df.columns, xaxis = df.columns)


def selection_fn(trace,points,selector):
    t.data[0].cells.values = [df.loc[points.point_inds][col] for col in varlist]



scatter.on_selection(selection_fn)

##Put everything together
VBox((HBox(axis_dropdowns.children),f,t))




In [ ]:
major_axis = np.percentile(data['Cells_AreaShape_MajorAxisLength'],.99)
minor_axis = np.percentile(data['Cells_AreaShape_MinorAxisLength'],.99)



In [ ]:
def bounding_box(x, y, minor_axis, major_axis):
    
    
    minor_radius = round(minor_axis/2, 3) * 3
    major_radius = round(major_axis/2, 3) * 3
    
    xmin = x - minor_radius
    xmax = x + minor_radius
    ymin = y - major_radius
    ymax = y + major_radius


    width, height = xmax - xmin, ymax - ymin
    
    return int(ymin), int(ymax), int(xmin), int(xmax)




def plotting_cropped_image_2(imagelist, labels, boundingbox):
    
    '''imagelist: should be list of image path, 
      labels: list of channels,
     boundingbox: Calculated in a function required to crop images
     image_cropping: dimesion used [ymin:ymax, xmin, xmax ]'''
       

    fig = plt.figure(figsize=(15,15))
    

    for i, im, lab in zip(range(1,6), imagelist, labels):
        
    
        image = skimage.io.imread(im)

        cropped = image[bbox[0]: bbox[1], bbox[2]: bbox[3]]
    
#     # cropping [ymin : ymax, xmin:xmax]
        ax = fig.add_subplot(1,5,i)
        ax.imshow(cropped, cmap='gray')
        ax.set_title(lab)
        #plt.savefig(os.path.join(outpath, 'weired_datapoint'))
    
    return ax


major_axis = round(np.percentile(data['Cells_AreaShape_MajorAxisLength'],.99)/2 * 5, 4)
minor_axis = round(np.percentile(data['Cells_AreaShape_MinorAxisLength'],.99)/2 * 5, 4)


col = [e for e in t.data[0].header.values]
dt = [list(i) for i in zip(*t.data[0].cells.values)]
dtt = pd.DataFrame(dt, columns = col)


for i, cpname, w, s, lx, ly in zip(range(len(dtt)),
                                dtt['Metadata_compound_name'], 
                                dtt['Metadata_Well'], 
                                dtt['Metadata_Site'],
                                dtt['Cells_Location_Center_X'], 
                                dtt['Cells_Location_Center_Y']):

    

    
    tmp = (data[(data["Metadata_compound_name"] == str(cpname)) &
        (data['Cells_Location_Center_X'] == lx) 
        & (data['Cells_Location_Center_Y'] == ly) 
        & (data['Metadata_Well'] == str(w))
       & (data['Metadata_Site'] == s)]
           .reset_index()
          )
    
   


    
    imagelist = tmp.loc[:, ['FileName_OrigDNA', 
                        "FileName_OrigRNA", 
                        "FileName_OrigMito", 
                        "FileName_OrigAGP",
                        "FileName_OrigER"]].values.tolist()[0]


    input_dir = tmp['PathName_OrigDNA'][0]
    
    imlist = sorted([os.path.join(input_dir, img) for img in imagelist])

    labels = ['Ch1: Mito', 'Ch2: AGP', 'Ch3: RNA', 'Ch4: ER', 'Ch5: DNA']



    bbox = bounding_box(lx, ly, minor_axis, major_axis)
    image_plot = plotting_cropped_image_2(imlist, labels, bbox)
    plt.savefig(os.path.join(outpath, str(i)))
    
    
    
    

